In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
# for plotting
import seaborn as sns
sns.set(style='white', font_scale=1.5)
#for working with Bam files
import pysam
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
#for distance
from scipy.spatial import distance
import datetime
from mpl_toolkits.mplot3d import Axes3D

import logging
from gridmap import grid_map
import os

/broad/blainey_lab/blainey-ipython/packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# Step1:
## Collect the results from the SNP caller
## and modify it to get one big list of optional candidates.

In [2]:
cd /broad/mushroom_lasagna/YB/Ht115_new_samples

/broad/mushroom_lasagna/YB/Ht115_new_samples


In [3]:
################################################################
#create the header file _ by using the original output file
frames=[]
out_files = !ls | grep 'T34_N48.txt'
f=out_files[0]
print f
!grep contig {f} > /broad/mushroom_lasagna/YB/{'header2.txt'} 
################################################################

T34_N48.txt


In [4]:
#load all files into one big table - sort by 
#not_contain_the outer sample
frames=[]
out_files = !ls | grep 'T' | grep '.txt' | grep -v '37'
count=1
columns = pd.read_csv('/broad/mushroom_lasagna/YB/header.txt', sep='\t', comment='#').columns
for file in out_files:
    wt = pd.read_csv(file, sep='\t')
    wt = pd.DataFrame(wt.as_matrix(), columns=columns)
    print ("for: ",file," -The size is: ",wt.shape)
    if (count==1):
        wf=wt
    else:
        wf=wf.append(wt, ignore_index=True)
    count=count+1
print (wf.shape)

/broad/blainey_lab/blainey-ipython/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


('for: ', 'T34_N38.txt', ' -The size is: ', (4773, 51))
('for: ', 'T34_N44.txt', ' -The size is: ', (4422, 51))
('for: ', 'T34_N45.txt', ' -The size is: ', (5023, 51))
('for: ', 'T34_N47.txt', ' -The size is: ', (5093, 51))
('for: ', 'T34_N48.txt', ' -The size is: ', (4812, 51))
('for: ', 'T34_N49.txt', ' -The size is: ', (5029, 51))
('for: ', 'T34_N54.txt', ' -The size is: ', (5193, 51))
('for: ', 'T34_N56.txt', ' -The size is: ', (5154, 51))
('for: ', 'T34_N57.txt', ' -The size is: ', (4572, 51))
('for: ', 'T34_N63.txt', ' -The size is: ', (5130, 51))
('for: ', 'T38_N34.txt', ' -The size is: ', (3167, 51))
('for: ', 'T38_N44.txt', ' -The size is: ', (3352, 51))
('for: ', 'T38_N45.txt', ' -The size is: ', (2952, 51))
('for: ', 'T38_N47.txt', ' -The size is: ', (3475, 51))
('for: ', 'T38_N48.txt', ' -The size is: ', (2800, 51))
('for: ', 'T38_N49.txt', ' -The size is: ', (4001, 51))
('for: ', 'T38_N54.txt', ' -The size is: ', (3522, 51))
('for: ', 'T38_N56.txt', ' -The size is: ', (305

In [12]:
cwf=wf.drop_duplicates(['position','contig'])

In [11]:
f1=open('/broad/mushroom_lasagna/YB/Ht115_new_samples/log.txt', 'w+')
co=1
columns=('contig','position','context','ref_all','alt_all','t_lod_fstar','vector','distance_in_g1','distance_in_g2','per_bet_gr','real_bet_gr','ref_af','alt_af','type','full_data')
mutation=[]
f1.write("Log for run: /n")
keep=['contig', 'position','context', 'ref_allele','alt_allele','t_lod_fstar']
locations=cwf[keep]
print "full size of the list:",locations.shape[0]

full size of the list: 64770


# Get the names for all bams file for every sample

In [14]:
files = pd.read_csv('/broad/mushroom_lasagna/YB/Ht115_new_samples/new_HT_samples.csv')
print files.shape
files[0:13]

(13, 4)


,sample_id,individual_id,clean_bam_file_capture,short_letter_code
0,T37,HT115_6,/seq/picard_aggregation/G101103/HT6W37/v4/HT6W...,T
1,T45,HT115_6,/seq/picard_aggregation/G101103/HT6W45/v4/HT6W...,T
2,T47,HT115_6,/seq/picard_aggregation/G101103/HT6W47/v4/HT6W...,T
3,T48,HT115_6,/seq/picard_aggregation/G101103/HT6W48/v4/HT6W...,T
4,T49,HT115_6,/seq/picard_aggregation/G101103/HT6W49/v4/HT6W...,T
5,T54,HT115_6,/seq/picard_aggregation/G101103/HT6W54/v4/HT6W...,T
6,T34,HT115_6,/seq/picard_aggregation/G101103/HT6W34/v4/HT6W...,T
7,T63,HT115_6,/seq/picard_aggregation/G101103/HT6W63/v4/HT6W...,T
8,T56,HT115_6,/broad/blainey_lab_storage/YB/G101103/Second_B...,T
9,T38,HT115_6,/broad/blainey_lab_storage/YB/G101103/Second_B...,T


# For every SNV, collect all reads from the samples (bam files). 

In [ ]:
#
#and save the table - should run only once.
#def mat_from_bam(locations):
# this have to be in seperate line - ??????
#!cd /seq/picard_aggregation/G101103/
f1=open('/broad/mushroom_lasagna/YB/RPE_new_samples/log.txt', 'w+')
f1.write("Log for run: /n")

head=[]
begin=!pwd 
begin=begin.pop()

#prepare list of all bam files to read
#open bam file and load all the locations - and add to the table
for ind in files.index:
    head, tail =os.path.split(files.clean_bam_file_capture[ind])
    print head
    os.chdir(head)
    path=files.clean_bam_file_capture[ind]
    samfile = pysam.AlignmentFile(path, "rb")
    #Samples.append(samfile)
    #for every bam file run over all the mutations
    seq=[]
    f1.write("Reading from file: %s\n"%(tail))
    f1.flush()
    for i in locations.index:
        if (i%500 ==0):
            f1.write("file %s finished = %i lines\n" %(tail,i))
            f1.flush()
        s=""
        chr= str(locations.contig[i])
        region= int(float(locations.position[i]))
        region=region-1
        for row in samfile.pileup(chr,region,region+1):
                if row.pos ==region:
                    for row in row.pileups:
                        if (row.query_position is not None):
                            #print (row.query_position)
                            #print (row.alignment.query_sequence[row.query_position])
                            s+=row.alignment.query_sequence[row.query_position]
        s=''.join(sorted(s.upper()))
        seq.append(s)
        # add the seq as column in the db - location
    #locations[f]=pd.Series(seq,index=f)
    ntail = tail.split('.')[0]
    #print "ntail= ",ntail," len seq= ",len(seq)
    locations[ntail]=seq

########################
#and save the new table
locations.to_pickle('/broad/mushroom_lasagna/YB/HT11_new_samples/mutations.pkl')
f1.write("file %s finished = %i lines\n" %(ntail,i))
f1.flush()
f1.write("finito")
f1.close()

In [17]:
locations[0:3]

,contig,position,context,ref_allele,alt_allele,t_lod_fstar,HT6W37,HT6W45,HT6W47,HT6W48,HT6W49,HT6W54,HT6W34,HT6W63,HT115_42112,HT115_42221,HT115_5211,HT115_7221,HT115
0,1,1131218,TCTxCCT,C,T,7.95299,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCC,ACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCC
1,1,1531286,ACAxAGA,C,G,6.83966,ACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGG,CCCCCCCCCCCCCCCCCCCCCCCCGGGGGGGG,AACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGGG,CCCCCCCCCCCCCCCCCCCCCCCCCGGGG,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGT,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGGGG,CCCCCCCCCCCCCCCCCCCCCCCCGGGGGG,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGGGGGGG,AACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGG,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGG,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGG,CCCCCCCCCCCCCCCCCCCCCCCCCCCCG
2,1,1894948,AAAxAAA,C,A,7.7051,AAAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCG,AAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCG,ACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AAAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCG,AACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AAAAAAAAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,AAAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,ACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCC


In [18]:
#For HT115:
#rename all the samples you are going to analyse

print locations.shape
#locations =locations.drop('HT115', 1)
locations=locations.rename(columns={"HT115":"HT_CCLE"})
locations=locations.rename(columns={"HT115_42112":"HT6W56"})
locations=locations.rename(columns={"HT115_42221":"HT6W38"})
locations=locations.rename(columns={"HT115_5211":"HT6W44"})
locations=locations.rename(columns={"HT115_7221":"HT6W57"})
col=locations.columns
imp= filter(lambda x: 'HT' in x, col)
print imp
locations[0:3]

(64770, 19)
['HT6W37', 'HT6W45', 'HT6W47', 'HT6W48', 'HT6W49', 'HT6W54', 'HT6W34', 'HT6W63', 'HT6W56', 'HT6W38', 'HT6W44', 'HT6W57', 'HT_CCLE']


,contig,position,context,ref_allele,alt_allele,t_lod_fstar,HT6W37,HT6W45,HT6W47,HT6W48,HT6W49,HT6W54,HT6W34,HT6W63,HT6W56,HT6W38,HT6W44,HT6W57,HT_CCLE
0,1,1131218,TCTxCCT,C,T,7.95299,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCC,ACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCCCCCCCCC
1,1,1531286,ACAxAGA,C,G,6.83966,ACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGG,CCCCCCCCCCCCCCCCCCCCCCCCGGGGGGGG,AACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGGG,CCCCCCCCCCCCCCCCCCCCCCCCCGGGG,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGT,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGGGG,CCCCCCCCCCCCCCCCCCCCCCCCGGGGGG,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGGGGGGG,AACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGG,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGG,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGG,CCCCCCCCCCCCCCCCCCCCCCCCCCCCG
2,1,1894948,AAAxAAA,C,A,7.7051,AAAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCG,AAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCG,ACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AAAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCG,AACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AAAAAAAAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,AAAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,AAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,ACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCCCCCC


In [ ]:
### For RPE1:
#print locations.shape

#locations=locations.rename(columns={"RPE1_7212":"RPE5W44"})
#locations=locations.rename(columns={"RPE1_6222":"RPE5W28"})
#locations=locations.rename(columns={"RPE1_6221":"RPE5W32"})
#locations=locations.rename(columns={"RPE1_621":"RPE5W36"})
#locations=locations.rename(columns={"RPE1_42221":"RPE5W38"})

#col=locations.columns
#imp= filter(lambda x: 'RPE' in x, col)
#print imp
#locations[0:3]

# Preapare the dataframe of the full list of candidates SNVs.

In [19]:
#this function convert one 'spot' from the full mutation table
def make_spot(mut,i):
    col=mut.columns
    imp= filter(lambda x: 'HT' in x, col)+filter(lambda x: '2nd' in x, col)
    seq=[]
    for f in imp:
        seq.append(mut[f][i])
    samples=pd.DataFrame(seq,index=imp,columns=['Seq'])
    return samples

def pre_seq_bin(seq):
    #######################
    #threshold for the binary is 20% 
    thr=0.2
    binary=np.zeros(4)#'ATCG'
    count=np.array([seq.count('A'),seq.count('T'),seq.count('C'),seq.count('G')])
    total= len(seq)
    percent = (count.astype(float)/total)
    count=0
    for i in percent:
        if (i>thr):
            binary[count]=1
        count=count+1         
    return binary

#function that read seq and return matrix of counts for each letter
def print_counts(seq):
    count=np.array([seq.count('A'),seq.count('T'),seq.count('C'),seq.count('G')])
    return count

#function that read seq and return matrix of percentage for each letter
def pre_seq(seq):
    count=np.array([seq.count('A'),seq.count('T'),seq.count('C'),seq.count('G')])
    total= len(seq)
    percent = (count.astype(float)/total)
    return percent

#build the vector from nucleotides
def convert_letter(char):#'ATCG'
    if char=="A":
        ans=0
    elif char=="T":
        ans=1
    elif char=="C":
        ans=2
    elif char=="G":
        ans=3
    else:
        ans=-1
    return ans

#show specific lines
def present(mut,ind):
    for i in mut.loc[ind].full_data :
        print i[0],i[2],i[6]
    return

In [ ]:
#add "full_data" to the mutation table:
cwf=locations
fmut=[]
columns=('contig','position','context','ref_all','alt_all','t_lod_fstar','full_data')
for i1 in cwf.index:
    spot=make_spot(cwf,i1)
    count=0
    fsdata=[]
    for i2 in spot.Seq:
        #print (spot.index[count],": ",print_counts(i)," ", pre_seq_bin(i))
        sdata=(spot.index[count],": ",print_counts(i2)," ",pre_seq(i2)," ", pre_seq_bin(i2))
        fsdata.append(sdata)
        count=count+1
    line=(cwf.contig[i1],cwf.position[i1],cwf.context[i1],cwf.ref_allele[i1],cwf.alt_allele[i1],cwf.t_lod_fstar[i1],fsdata)
    fmut.append(line)
    #cwf.set_value(index=i1,col='full_data',value=fsdata)
full_mut = pd.DataFrame(fmut, columns=columns)
full_mut.to_pickle('/broad/mushroom_lasagna/YB/Ht115_new_samples/full_mutations.pkl')
print "finish next time can skip"